In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quick start with Model Garden - Path Foundation

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fgoogle-health%2Fpath-foundation%2Fmaster%2Fnotebooks%2Fquick_start_with_model_garden.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/google-health/path-foundation/blob/master/notebooks/quick_start_with_model_garden.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates deploying Path Foundation to Vertex AI and making online predictions to get embeddings from pathology image patches.

Vertex AI makes it easy to serve your model and make it accessible to the world. Learn more about [Vertex AI](https://cloud.google.com/vertex-ai/docs/start/introduction-unified-platform).


### Objective

- Deploy Path Foundation to a Vertex AI Endpoint and get online predictions

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

In [ ]:
# @title Import packages and define common functions

! pip install --upgrade --quiet 'ez-wsi-dicomweb>=6.0.5'

import datetime
import importlib
import os
import uuid
from typing import Iterator, Union

import ez_wsi_dicomweb.ml_toolkit.dicom_path as dicom_path
import matplotlib.pyplot as plt
import numpy as np
from ez_wsi_dicomweb import (credential_factory, dicom_slide,
                             dicom_web_interface, gcs_image, local_image,
                             patch_embedding, patch_embedding_endpoints,
                             patch_embedding_ensemble_methods)
from google.cloud import aiplatform

if not os.path.isdir("vertex-ai-samples"):
    ! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

common_util = importlib.import_module(
    "vertex-ai-samples.community-content.vertex_model_garden.model_oss.notebook_util.common_util"
)

models, endpoints = {}, {}


def render_patch(
    patch: Union[dicom_slide.DicomPatch, gcs_image.GcsPatch], plot_name: str = ""
) -> None:
    """Displays a patch from a DICOM slide or Cloud Storage image."""
    patch_bytes = patch.image_bytes()
    # Transforms monochrome imaging to three RGB channel representation.
    if len(patch_bytes.shape) == 2 or (
        len(patch_bytes.shape) == 3 and patch_bytes.shape[-1] == 1
    ):
        mem = np.zeros((224, 224, 3), dtype=patch_bytes.dtype)
        mem[..., np.arange(3)] = patch_bytes[...]
        patch_bytes = mem
    print(patch_bytes.shape)
    plt.figure(figsize=(2, 2))
    plt.imshow(patch_bytes)
    plt.title(plot_name)
    plt.axis("off")
    plt.show()

In [ ]:
# @title Set up Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. Make sure that you have the following required roles:
# @markdown - [Storage Admin](https://cloud.google.com/iam/docs/understanding-roles#storage.admin) (`roles/storage.admin`) to create and use Cloud Storage buckets
# @markdown - [Service Usage Admin](https://cloud.google.com/iam/docs/understanding-roles#serviceusage.serviceUsageAdmin) (`roles/serviceusage.serviceUsageAdmin`) to enable necessary APIs

# @markdown 3. Set up a [Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets).
# @markdown - A new bucket will be set up for you to store artifacts used in this notebook.
# @markdown - [Optional] To use an existing bucket, specify the `gs://` bucket URI. The specified Cloud Storage bucket should be located in the same region as where the notebook was launched. Note that a multi-region bucket (e.g. "us") is not considered a match for a single region covered by the multi-region range (e.g. "us-central1").

BUCKET_URI = ""  # @param {type:"string", placeholder:"[Optional] Cloud Storage bucket URI"}

# The region will be set automatically according to the Colab Enterprise environment.
REGION = ""


# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "path-foundation")


# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)


# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/storage.admin"
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/aiplatform.user"

## Use of EZ-WSI

This notebook leverages [EZ-WSI DICOMweb](https://github.com/GoogleCloudPlatform/EZ-WSI-DICOMweb), which makes it easier to work with DICOM data and generate embeddings from a variety of data sources.

## Get online predictions

In [ ]:
# @title #### Import deployed model

# @markdown To get [online predictions](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions), you will need a Path Foundation [Vertex AI Endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment) that has been deployed from Model Garden. If you have not already done so, go to the [Path Foundation model card](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/path-foundation) in Model Garden and click "Deploy" to deploy the model.

# @markdown This section instantiates a [V2PatchEmbeddingEndpoint class](https://github.com/GoogleCloudPlatform/EZ-WSI-DICOMweb/blob/0927ad2dcc73e5315f6af2bd4d022c3fe925e8bf/ez_wsi_dicomweb/patch_embedding_endpoints.py#L1369), which is an implementation of an abstraction interface through which EZ-WSI requests and receives embeddings. It defines a connection to the Path Foundation Vertex AI Endpoint and will be called to generate embeddings in the next sections.

# @markdown Fill in the endpoint ID and region below. You can find your deployed endpoint on the [Vertex AI online prediction page](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).

ENDPOINT_ID = ""  # @param {type: "string", placeholder:"e.g. 123456789"}
ENDPOINT_REGION = ""  # @param {type: "string", placeholder:"e.g. us-central1"}

endpoint = patch_embedding_endpoints.V2PatchEmbeddingEndpoint(
    project_id=PROJECT_ID,
    endpoint_location=ENDPOINT_REGION,
    endpoint_id=ENDPOINT_ID,
)

### Predict

You can send [online prediction requests](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions#predict-request) to the endpoint with image patches, cropped sub-regions of a digital pathology image, to generate embeddings.

The following examples illustrate how to use Path Foundation leveraging EZ-WSI to generate embeddings for a single patch, multiple patches, or a whole image from:

* A DICOM image stored in [Cloud Healthcare DICOM Store](https://cloud.google.com/healthcare-api/docs/how-tos/dicom)
* An image stored in [Cloud Storage](https://cloud.google.com/storage/docs)
* A local in-memory representation

#### Generate a single patch embedding

These examples demonstrate the simplest patch-to-embedding interface using EZ-WSI to generate an embedding for a single patch.

In [ ]:
# @title ##### From a DICOM image

# @markdown This section shows an example of generating a single patch embedding from a DICOM image stored in a Google-hosted DICOM store containing the [Camelyon16 dataset](https://camelyon16.grand-challenge.org/).
# @markdown To access this dataset, request access to our [research endpoint](https://developers.google.com/health-ai-developer-foundations/model-serving/research-endpoints).

# @markdown You can replace the fields below to use your own data.

# @markdown Click "Show Code" to see more details.

# Defines DICOM image stored within a Google DICOM store.
DATASET_PROJECT_ID = "hai-cd3-foundations"  # @param {type:"string", placeholder:"Project ID"}
DATASET_LOCATION = "us-west1"  # @param {type:"string", placeholder:"Cloud Healthcare dataset location"}
DATASET_ID = "pathology"  # @param {type:"string", placeholder:"Cloud Healthcare dataset ID"}
STORE_ID = "camelyon"  # @param {type:"string", placeholder:"DICOM store ID"}
STUDY_INSTANCE_UID = "1.3.6.1.4.1.11129.5.7.999.186491099540.79362771.1709051344594461"  # @param {type:"string", placeholder:"DICOM study instance UID"}
SERIES_INSTANCE_UID = "1.3.6.1.4.1.11129.5.7.999.186491099540.79362771.1709051344626463"  # @param {type:"string", placeholder:"DICOM series instance UID"}

# Full path to DICOM store and DICOM series containing whole slide imaging.
series_path = dicom_path.FromString(
    f"https://healthcare.googleapis.com/v1/projects/{DATASET_PROJECT_ID}/locations/{DATASET_LOCATION}/datasets/{DATASET_ID}/dicomStores/{STORE_ID}/dicomWeb/studies/{STUDY_INSTANCE_UID}/series/{SERIES_INSTANCE_UID}"
)

# Credential factory that provides EZ-WSI with credentials to access DICOM imaging metadata.
dcf = credential_factory.DefaultCredentialFactory()

# Create interface to slide, retrieves slide metadata but not slide imaging.
ds = dicom_slide.DicomSlide(
    path=series_path, dwi=dicom_web_interface.DicomWebInterface(dcf)
)

# Request a single patch of imaging from the highest magnfication.
patch = ds.get_patch(level=ds.native_level, x=43000, y=10000, width=224, height=224)

# Takes a patch (DicomPatch or GcsPatch) and returns an embedding.
embedding = patch_embedding.get_patch_embedding(endpoint, patch)

# Display image (optional, for illustrating the source imaging for the embedding)
render_patch(patch)

# Display first 12 values in the embedding
print(embedding[:12])

In [ ]:
# @title ##### From an image in Cloud Storage

# @markdown This section shows an example of generating a single patch embedding from an image stored in Cloud Storage.

# @markdown You can replace `GCS_URI` below to use your own data.

# @markdown Click "Show Code" to see more details.

GCS_URI = "gs://healthai-us/pathology/example_large_patch.jpeg"  # @param {type:"string", placeholder:"Cloud Storage file URI"}

# Create a reference to an image stored on Cloud Storage
# Authenticates with default credentials by default.
image = gcs_image.GcsImage(
    GCS_URI, credential_factory=credential_factory.NoAuthCredentialsFactory()
)

# Define coordinates of image patch
patch = image.get_patch(x=10, y=10, width=224, height=224)

# Takes a patch (DicomPatch or GcsPatch) and returns an embedding.
embedding = patch_embedding.get_patch_embedding(endpoint, patch)

# Display image (optional, for illustrating the source imaging for the embedding)
render_patch(patch)

# Display first 12 values in the embedding
print(embedding[:12])

In [ ]:
# @title ##### From local in-memory data

# @markdown This section shows an example of generating a single patch embedding from an in-memory NumPy array.

# @markdown Click "Show Code" to see more details.

# Create an in-memory uncompressed image
memory = np.zeros((224, 224, 3), dtype=np.uint8)

# Construct an image from the in memory patch
image = local_image.LocalImage(memory)

# Define coordinates of image patch
patch = image.get_patch(x=0, y=0, width=224, height=224)

# Takes a patch (DicomPatch or GcsPatch) and returns an embedding.
embedding = patch_embedding.get_patch_embedding(endpoint, patch)

# Display image (optional, for illustrating the source imaging for the embedding)
render_patch(patch, "Image is expected to be entirely black")

# Display first 12 values in the embedding
# Embeddings will match previous same image bytes
# but from local sourceembeddings.
print(embedding[:12])

#### Generate multiple patch embeddings

EZ-WSI provides general purpose interfaces that greatly reduce the time for generating embeddings for multiple patches.

In [ ]:
# @title ##### From a DICOM image

# @markdown This section shows an example of generating embeddings for multiple patches from a DICOM image stored in a Google-hosted DICOM store containing the [Camelyon16 dataset](https://camelyon16.grand-challenge.org/).
# @markdown To access this dataset, request access to our [research endpoint](https://developers.google.com/health-ai-developer-foundations/model-serving/research-endpoints).

# @markdown You can replace the fields below to use your own data.

# @markdown Click "Show Code" to see more details.

# Defines DICOM image stored within a Google DICOM store.
DATASET_PROJECT_ID = "hai-cd3-foundations"  # @param {type:"string", placeholder:"Project ID"}
DATASET_LOCATION = "us-west1"  # @param {type:"string", placeholder:"Cloud Healthcare dataset location"}
DATASET_ID = "pathology"  # @param {type:"string", placeholder:"Cloud Healthcare dataset ID"}
STORE_ID = "camelyon"  # @param {type:"string", placeholder:"DICOM store ID"}
STUDY_INSTANCE_UID = "1.3.6.1.4.1.11129.5.7.999.186491099540.79362771.1709051344594461"  # @param {type:"string", placeholder:"DICOM study instance UID"}
SERIES_INSTANCE_UID = "1.3.6.1.4.1.11129.5.7.999.186491099540.79362771.1709051344626463"  # @param {type:"string", placeholder:"DICOM series instance UID"}


def patch_generator(
    ds: dicom_slide.DicomSlide,
    level: dicom_slide.Level,
    x: int,
    y: int,
    step: int,
    num_patches: int,
) -> Iterator[dicom_slide.DicomPatch]:
    """Generates sequential patches at a pyramid level of a DICOM slide."""
    for _ in range(num_patches):
        yield ds.get_patch(level, x, y, 224, 224)
        x += step
        if x + step >= level.width:
            x = 0
            y += step


# Full path to DICOM store and DICOM series containing whole slide imaging.
series_path = dicom_path.FromString(
    f"https://healthcare.googleapis.com/v1/projects/{DATASET_PROJECT_ID}/locations/{DATASET_LOCATION}/datasets/{DATASET_ID}/dicomStores/{STORE_ID}/dicomWeb/studies/{STUDY_INSTANCE_UID}/series/{SERIES_INSTANCE_UID}"
)

# Credential factory that provides EZ-WSI with credentials to access DICOM imaging metadata.
dcf = credential_factory.DefaultCredentialFactory()

# Create interface to slide, retrieves slide metadata but not slide imaging.
ds = dicom_slide.DicomSlide(
    path=series_path, dwi=dicom_web_interface.DicomWebInterface(dcf)
)

# Generate 500 patches sampled across the a single pyramid.
# Note, the generator could return patches sampled across multiple pyramid
# layers or images, the only requirement is patches from the same source image
# or pyramid layer are clustered.
patches = patch_generator(ds, ds.native_level, 43000, 10000, 224, 500)

# Takes the patches (DicomPatch or GcsPatch) and returns embeddings.
embeddings = patch_embedding.generate_patch_embeddings(endpoint, patches)

# Convert the embedding generator into a list of values.
embeddings = list(embeddings)

# Display total number of embeddings generated
print(f"Embeddings returned: {len(embeddings)}")

# Display results for first two embeddings returned
print("First two embeddings results")
for result in embeddings[:2]:
    # render the source embedding patch
    render_patch(result.patch)

    print(
        f"Patch, Location x: {result.patch.x} y: {result.patch.y}; Dimensions width: {result.patch.width} height: {result.patch.height}"
    )

    print("First 12 values of patch image embedding.")
    print(result.embedding[:12])

In [ ]:
# @title ##### From an image in Cloud Storage

# @markdown This section shows an example of generating embeddings for multiple patches from an image stored in Cloud Storage.

# @markdown You can replace `GCS_URI` below to use your own data.

# @markdown Click "Show Code" to see more details.

GCS_URI = "gs://healthai-us/pathology/example_large_patch.jpeg"  # @param {type:"string", placeholder:"Cloud Storage file URI"}


def patch_generator(
    image: gcs_image.GcsImage, x: int, y: int, step: int, num_patches: int
) -> Iterator[gcs_image.GcsPatch]:
    """Generates sequential patches at a pyramid level of a DICOM slide."""
    for _ in range(num_patches):
        yield image.get_patch(x, y, 224, 224)
        x += step
        if x + 224 >= image.width:
            x = 0
            y += step


# Create a reference to an image stored on Cloud Storage
# Authenticates with default credentials by default.
image = gcs_image.GcsImage(
    GCS_URI, credential_factory=credential_factory.NoAuthCredentialsFactory()
)

# The generator, generates 500 patches sampled across the Cloud Storage image.
# Since the image is relatively small, we are generating overlapping embeddings
# by stepping the patches 10 pixels at a time.
patches = patch_generator(image, 0, 0, 10, 500)

# Takes the patches (DicomPatch or GcsPatch) and returns embeddings.
embeddings = patch_embedding.generate_patch_embeddings(endpoint, patches)

# Convert the embedding generator into a list of values.
embeddings = list(embeddings)

# Display total number of embeddings generated
print(f"Embeddings returned: {len(embeddings)}")

# Display results for two embeddings returned
print("Embeddings results")
for result in (embeddings[0], embeddings[20]):
    # render the source embedding patch
    render_patch(result.patch)

    print(
        f"Patch, Location x: {result.patch.x} y: {result.patch.y}; Dimensions width: {result.patch.width} height: {result.patch.height}"
    )

    print("First 12 values of patch image embedding.")
    print(result.embedding[:12])

In [ ]:
# @title ##### From local in-memory data

# @markdown This section shows an example of generating embeddings for multiple patches from an in-memory NumPy array.

# @markdown Click "Show Code" to see more details.


def patch_generator(
    image: local_image.LocalImage, x: int, y: int, step: int, num_patches: int
) -> Iterator[gcs_image.GcsPatch]:
    """Generates sequential patches at a pyramid level of a DICOM slide."""
    for _ in range(num_patches):
        yield image.get_patch(x, y, 224, 224)
        x += step
        if x + 224 >= image.width:
            x = 0
            y += step


# Create an in-memory uncompressed image filled with random noise.
memory = np.random.randint(0, high=255, size=(224 * 50, 224, 3), dtype=np.uint8)

# Construct an image from the in memory patch
image = local_image.LocalImage(memory)

# The generator, generates 50 patches sampled across the image.
# The image is relatively small so were generating embeddings for overlapping
# patches, stepping the patches 10 pixels at a time.
patches = patch_generator(image, 0, 0, 224, 50)

# Takes the patches (DicomPatch or GcsPatch) and returns embeddings.
embeddings = patch_embedding.generate_patch_embeddings(endpoint, patches)

# Convert the embedding generator into a list of values.
embeddings = list(embeddings)

# Display total number of embeddings generated
print(f"Embeddings returned: {len(embeddings)}")

# Display results for two embeddings returned
print("Embeddings results")
for result in (embeddings[0], embeddings[20]):
    # render the source embedding patch
    render_patch(result.patch, "Random Colors Expected")

    print(
        f"Patch, Location x: {result.patch.x} y: {result.patch.y}; Dimensions width: {result.patch.width} height: {result.patch.height}"
    )

    print("First 12 values of patch image embedding.")
    print(result.embedding[:12])

#### Generate whole image embeddings

EZ-WSI contains high level patch generation functions to selectively generate patches from regions of interest (e.g. areas containing tissue) within a whole image.

In [ ]:
# @title ##### From a DICOM image

# @markdown This section shows an example of generating embeddings that are selectively sampled across a DICOM image stored in a Google-hosted DICOM store containing the [Camelyon16 dataset](https://camelyon16.grand-challenge.org/).
# @markdown To access this dataset, request access to our [research endpoint](https://developers.google.com/health-ai-developer-foundations/model-serving/research-endpoints).

# @markdown You can replace the fields below to use your own data.

# @markdown Click "Show Code" to see more details.

# Defines DICOM image stored within a Google DICOM store.
DATASET_PROJECT_ID = "hai-cd3-foundations"  # @param {type:"string", placeholder:"Project ID"}
DATASET_LOCATION = "us-west1"  # @param {type:"string", placeholder:"Cloud Healthcare dataset location"}
DATASET_ID = "pathology"  # @param {type:"string", placeholder:"Cloud Healthcare dataset ID"}
STORE_ID = "camelyon"  # @param {type:"string", placeholder:"DICOM store ID"}
STUDY_INSTANCE_UID = "1.3.6.1.4.1.11129.5.7.999.186491099540.79362771.1709051344594461"  # @param {type:"string", placeholder:"DICOM study instance UID"}
SERIES_INSTANCE_UID = "1.3.6.1.4.1.11129.5.7.999.186491099540.79362771.1709051344626463"  # @param {type:"string", placeholder:"DICOM series instance UID"}

# Full path to DICOM store and DICOM series containing whole slide imaging.
series_path = dicom_path.FromString(
    f"https://healthcare.googleapis.com/v1/projects/{DATASET_PROJECT_ID}/locations/{DATASET_LOCATION}/datasets/{DATASET_ID}/dicomStores/{STORE_ID}/dicomWeb/studies/{STUDY_INSTANCE_UID}/series/{SERIES_INSTANCE_UID}"
)

# Credential factory that provides EZ-WSI with credentials to access DICOM imaging metadata.
dcf = credential_factory.DefaultCredentialFactory()

# Create interface to slide, retrieves slide metadata but not slide imaging.
ds = dicom_slide.DicomSlide(
    path=series_path, dwi=dicom_web_interface.DicomWebInterface(dcf)
)

# Optional but highly recommended, enables DS to retrieve patch imaging more
# efficiently when generating the tissue mask.
ds.init_slide_frame_cache()

embeddings = patch_embedding.get_dicom_image_embeddings(endpoint, ds, ds.native_level)

# Display total number of embeddings generated
print(f"Embeddings returned: {len(embeddings)}")

# Display results for first two embeddings returned
print("First two embeddings results")
for result in embeddings[:2]:
    # render the source embedding patch
    render_patch(result.patch)

    print(
        f"Patch, Location x: {result.patch.x} y: {result.patch.y}; Dimensions width: {result.patch.width} height: {result.patch.height}"
    )

    print("First 12 values of patch image embedding.")
    print(result.embedding[:12])

In [ ]:
# @title ##### From an image in Cloud Storage

# @markdown This section shows an example of generating embeddings that are selectively sampled from an image stored in Cloud Storage.

# @markdown You can replace `GCS_URI` below to use your own data.

# @markdown Click "Show Code" to see more details.

GCS_URI = "gs://healthai-us/pathology/example_large_patch.jpeg"  # @param {type:"string", placeholder:"Cloud Storage file URI"}

# Create a reference to an image stored on Google Cloud Storage
# Authenticates with default credentials by default.
image = gcs_image.GcsImage(
    GCS_URI, credential_factory=credential_factory.NoAuthCredentialsFactory()
)

embeddings = patch_embedding.get_gcs_image_embeddings(endpoint, image)

# Display total number of embeddings generated
print(f"Embeddings returned: {len(embeddings)}")

# Display results for first two embeddings returned
print("First two embeddings results")
for result in embeddings[:2]:
    # render the source embedding patch
    render_patch(result.patch)

    print(
        f"Patch, Location x: {result.patch.x} y: {result.patch.y}; Dimensions width: {result.patch.width} height: {result.patch.height}"
    )

    print("First 12 values of patch image embedding.")
    print(result.embedding[:12])

In [ ]:
# @title ##### From local in-memory data

# @markdown This section shows an example of generating embeddings that are selectively sampled from an in-memory NumPy array.

# @markdown Click "Show Code" to see more details.

# Create an in-memory uncompressed image filled with random noise.
memory = np.random.randint(0, high=255, size=(224 * 50, 224, 3), dtype=np.uint8)

# Construct an image from the in memory patch
image = local_image.LocalImage(memory)

embeddings = patch_embedding.get_gcs_image_embeddings(endpoint, image)

# Display total number of embeddings generated
print(f"Embeddings returned: {len(embeddings)}")

# Display results for first two embeddings returned
print("First two embeddings results")
for result in embeddings[:2]:
    # render the source embedding patch
    render_patch(result.patch)

    print(
        f"Patch, Location x: {result.patch.x} y: {result.patch.y}; Dimensions width: {result.patch.width} height: {result.patch.height}"
    )

    print("First 12 values of patch image embedding.")
    print(result.embedding[:12])

In [ ]:
# @title ##### Reduce embedding results to a single embedding

# @markdown The selectively generated embeddings for a whole image can be reduced into a single embedding using a utility function.

# @markdown This section shows an example to return the mean embedding result of all the embeddings in the results sequence.

# @markdown Click "Show Code" to see more details.

image = gcs_image.GcsImage(
    "gs://healthai-us/pathology/example_large_patch.jpeg",
    credential_factory=credential_factory.NoAuthCredentialsFactory(),
)
embeddings = patch_embedding.get_gcs_image_embeddings(endpoint, image)

print(f"Reducing {len(embeddings)} to a single embedding.")
# Reduces the 20 embeddings returned by `get_gcs_image_embeddings` to a single
# embedding.
embedding = patch_embedding_ensemble_methods.mean_patch_embedding(embeddings)

print("First 12 values of patch image embedding.")
print(embedding[:12])

## Next steps

Explore the other [notebooks](https://github.com/google-health/path-foundation/blob/master/notebooks) to learn what else you can do with the model.

## Clean up resources

In [ ]:
# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME